# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [15]:
%reload_ext dotenv
%dotenv

In [16]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os
from glob import glob

#Load price data
price_data_path = os.getenv("PRICE_DATA")

#Find path of parquet files
parquet_files = glob(os.path.join('/Users/yasssyasmine/DSI Certificate/production/05_src/data/prices/*/*.parquet/part.0.parquet'))


In [18]:
price_data_path

'../../../05_src/data/prices/'

In [19]:
len(parquet_files)

11207

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [20]:
import numpy as np

In [29]:
#Assign paraquet files to dd_px
dd_pn = dd.read_parquet(parquet_files).set_index("ticker")
dd_pn.head()


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463028,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142933,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652885,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219181,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237442,2819626,Health Care,Life Sciences Tools & Services,2000


In [30]:
#Add close and Adj_Close variables, returns based on Adjusted Close and daily High Low Range
dd_var = (dd_pn.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(AdjClose_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: (x ['Adj Close']/x['AdjClose_lag'] - 1)
).assign(
    hi_lo_range = lambda x: x['High']-x['Low'])

)

/var/folders/tb/6yq6bsrd7dsc3czlt3tbm_xm0000gn/T/ipykernel_93513/1461574749.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_var = (dd_pn.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [31]:
#Convert to pandas data frame
pd_feat = dd_var.compute()

#Add rolling average calc
pd_feat['rolling_10_mean'] = pd_feat['returns'].rolling(10).mean()


In [33]:
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,AdjClose_lag,returns,hi_lo_range,rolling_10_mean
ticker,,,,,,,,,,,,,,
HUM,2003-01-02,10.100000,10.350000,10.030000,10.310000,9.205752,856400,Health Care,Managed Health Care,2003,NaN,NaN,0.320001,NaN
HUM,2003-01-03,10.320000,10.480000,10.310000,10.400000,9.286115,795300,Health Care,Managed Health Care,2003,9.205752,0.008730,0.169999,NaN
HUM,2003-01-06,10.480000,10.500000,10.260000,10.320000,9.214682,962000,Health Care,Managed Health Care,2003,9.286115,-0.007692,0.240000,NaN
HUM,2003-01-07,10.260000,10.400000,10.250000,10.310000,9.205752,980000,Health Care,Managed Health Care,2003,9.214682,-0.000969,0.150000,NaN
HUM,2003-01-08,10.320000,10.440000,10.250000,10.400000,9.286115,1071600,Health Care,Managed Health Care,2003,9.205752,0.008730,0.190000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2021-12-27,34.230000,35.160000,34.220001,35.110001,32.587700,2857400,Information Technology,Communications Equipment,2021,31.678102,0.028714,0.939999,0.003317
JNPR,2021-12-28,35.099998,35.549999,35.080002,35.430000,32.884712,3582300,Information Technology,Communications Equipment,2021,32.587700,0.009114,0.469997,0.004199
JNPR,2021-12-29,35.450001,35.910000,35.349998,35.770000,33.200287,2243800,Information Technology,Communications Equipment,2021,32.884712,0.009596,0.560001,0.006599


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?


+ Would it have been better to do it in Dask? Why?


(1 pt)

1. No, it wasn't necessary to convert to panda to do the calculations as it is possible to do the rolling mean calclation in Dask. Also it would save us time from converting the dataframe to pandas. 

2. It may have been better to perform the calculation in Dask because Dask can be faster when handling larger datasets

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.